In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pyspark
import json

In [3]:
topics = ['testing-testing']
broker = 'kafka:9092'

In [4]:
sc = SparkContext(appName="app")
sc.setLogLevel("WARN")
sc

<SparkContext master=local[*] appName=app>

In [5]:
ssc = StreamingContext(sc, 10)
ssc

In [6]:
streaming = KafkaUtils.createDirectStream(ssc, topics, {"metadata.broker.list": broker})

In [7]:
count_this_batch = streaming.count().map(lambda x:('This batch: %s' % x))
count_this_batch.pprint()

In [8]:
def parse(v):
    return json.loads(v[1])

parsed = streaming.map(parse)
parsed.pprint(2)

In [9]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [10]:
maxlen = 50
UNK = 3

with open('dictionary-test.json', 'r') as fopen:
    dic = json.load(fopen)

def classify(sentences):
    
    def load_graph(frozen_graph_filename):
        with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def)
        return graph

    g = load_graph('frozen_model.pb')

    label = ['negative', 'positive']

    X = g.get_tensor_by_name('import/Placeholder:0')
    Y = g.get_tensor_by_name('import/logits:0')
    sess = tf.InteractiveSession(graph = g)
    
    sentences = sentences.collect()
    sentences = [row['word'] for row in sentences]
    
    x = np.zeros((len(sentences), maxlen))
    for i, sentence in enumerate(sentences):
        for no, k in enumerate(sentence.split()[:maxlen][::-1]):
            x[i, -1 - no] = dic.get(k,UNK)
    indices = np.argmax(sess.run(Y, feed_dict = {X: x}), axis = 1)
    
    # you can push to any database
    print([label[i] for i in indices])

parsed.foreachRDD(classify)

In [ ]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2019-12-31 10:00:50
-------------------------------------------
This batch: 24

-------------------------------------------
Time: 2019-12-31 10:00:50
-------------------------------------------
{'word': 'damn ted you must think gods a pussy', 'datetime': '2019-12-31 10:00:39.973912'}
{'word': 'ouch dont hold back tell us how you really feel', 'datetime': '2019-12-31 10:00:44.993906'}
...

['positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive']
-------------------------------------------
Time: 2019-12-31 10:01:00
-------------------------------------------
This batch: 19

-------------------------------------------
Time: 2019-12-31 10:01:00
-------------------------------------------
{'word': 'insane like a f

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


['negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive']
-------------------------------------------
Time: 2019-12-31 10:01:10
-------------------------------------------
This batch: 20

-------------------------------------------
Time: 2019-12-31 10:01:10
-------------------------------------------
{'word': 'that of course is the debt star', 'datetime': '2019-12-31 10:01:00.053094'}
{'word': 'reid and reasons share part of a syllable cant be coincidence', 'datetime': '2019-12-31 10:01:05.075464'}
...

['negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative']
